In [17]:
import eikon as ek
import pandas as pd
from datetime import datetime
import numpy as np
import os
import refinitiv.dataplatform.eikon as ek
import refinitiv.data as rd
import math
ek.set_app_key(os.getenv('EIKON_APIBZ'))

In [15]:
def query_refinitiv(asset = 'IVV', start_date = '2020-01-01', end_date = '2023-04-13'):

    # Retrieve the pricing data for the asset and date range
    prices, prc_err = ek.get_data(
        instruments=asset,
        fields=[
            'TR.OPENPRICE(Adjusted=0)',
            'TR.HIGHPRICE(Adjusted=0)',
            'TR.LOWPRICE(Adjusted=0)',
            'TR.CLOSEPRICE(Adjusted=0)',
            'TR.PriceCloseDate'
        ],
        parameters={
            'SDate': start_date,
            'EDate': end_date,
            'Frq': 'D'
        }
    )

    # Rename the columns of the DataFrame to more descriptive names
    prices.rename(
        columns={
            'TR.OPENPRICE(Adjusted=0)': 'open',
            'TR.HIGHPRICE(Adjusted=0)': 'high',
            'TR.LOWPRICE(Adjusted=0)': 'low',
            'TR.CLOSEPRICE(Adjusted=0)': 'close',
            'TR.PriceCloseDate': 'date'
        },
        inplace=True
    )

    # Convert the date column to a pandas datetime format
    prices['Date'] = pd.to_datetime(prices['Date'])

    # Format the date column as strings in the format "YYYY-MM-DD"
    prices['Date'] = prices['Date'].dt.strftime('%Y-%m-%d')
    return prices

In [7]:
def render_blotter(prices):
    ivv_prc = prices
    ivv_prc['Date'] = pd.to_datetime(ivv_prc['Date']).dt.date
    ivv_prc.drop(columns='Instrument', inplace=True)


    ##### Get the next business day from Refinitiv!!!!!!!
    rd.open_session()

    next_business_day = rd.dates_and_calendars.add_periods(
        start_date= ivv_prc['Date'].iloc[-1].strftime("%Y-%m-%d"),
        period="1D",
        calendars=["USA"],
        date_moving_convention="NextBusinessDay",
    )

    rd.close_session()
    ######################################################
    # Parameters:
    alpha1 = -0.01
    n1 = 3
    alpha2 = 0.01
    n2 = 5

    # submitted entry orders
    submitted_entry_orders = pd.DataFrame({
        "trade_id": range(1, ivv_prc.shape[0]),
        "date": list(pd.to_datetime(ivv_prc["Date"].iloc[1:]).dt.date),
        "asset": "IVV",
        "trip": 'ENTER',
        "action": "BUY",
        "type": "LMT",
        "price": round(
            ivv_prc['Close Price'].iloc[:-1] * (1 + alpha1),
            2
        ),
        'status': 'SUBMITTED'
    })

    # if the lowest traded price is still higher than the price you bid, then the
    # order never filled and was cancelled.
    with np.errstate(invalid='ignore'):
        cancelled_entry_orders = submitted_entry_orders[
            np.greater(
                ivv_prc['Low Price'].iloc[1:][::-1].rolling(n1).min()[
                ::-1].to_numpy(),
                submitted_entry_orders['price'].to_numpy()
            )
        ].copy()
    cancelled_entry_orders.reset_index(drop=True, inplace=True)
    cancelled_entry_orders['status'] = 'CANCELLED'
    cancelled_entry_orders['date'] = pd.DataFrame(
        {'cancel_date': submitted_entry_orders['date'].iloc[(n1-1):].to_numpy()},
        index=submitted_entry_orders['date'].iloc[:(1-n1)].to_numpy()
    ).loc[cancelled_entry_orders['date']]['cancel_date'].to_list()

    filled_entry_orders = submitted_entry_orders[
        submitted_entry_orders['trade_id'].isin(
            list(
                set(submitted_entry_orders['trade_id']) - set(
                    cancelled_entry_orders['trade_id']
                )
            )
        )
    ].copy()
    filled_entry_orders.reset_index(drop=True, inplace=True)
    filled_entry_orders['status'] = 'FILLED'
    for i in range(0, len(filled_entry_orders)):

        idx1 = np.flatnonzero(
            ivv_prc['Date'] == filled_entry_orders['date'].iloc[i]
        )[0]

        ivv_slice = ivv_prc.iloc[idx1:(idx1+n1)]['Low Price']

        fill_inds = ivv_slice <= filled_entry_orders['price'].iloc[i]

        if (len(fill_inds) < n1) & (not any(fill_inds)):
            filled_entry_orders.at[i, 'status'] = 'LIVE'
        else:
            filled_entry_orders.at[i, 'date'] = ivv_prc['Date'].iloc[
                fill_inds.idxmax()
            ]

    live_entry_orders = pd.DataFrame({
        "trade_id": ivv_prc.shape[0],
        "date": pd.to_datetime(next_business_day).date(),
        "asset": "IVV",
        "trip": 'ENTER',
        "action": "BUY",
        "type": "LMT",
        "price": round(ivv_prc['Close Price'].iloc[-1] * (1 + alpha1), 2),
        'status': 'LIVE'
    },
        index=[0]
    )

    if any(filled_entry_orders['status'] =='LIVE'):
        live_entry_orders = pd.concat([
            filled_entry_orders[filled_entry_orders['status'] == 'LIVE'],
            live_entry_orders
        ])
        # "today" is the next business day after the last closing price
        live_entry_orders['date'] = pd.to_datetime(next_business_day).date()

    filled_entry_orders = filled_entry_orders[
        filled_entry_orders['status'] == 'FILLED'
        ]

    entry_orders = pd.concat(
        [
            submitted_entry_orders,
            cancelled_entry_orders,
            filled_entry_orders,
            live_entry_orders
        ]
    ).sort_values(["date", 'trade_id'])


    # for every filled entry order, there must exist a submitted exit order:
    submitted_exit_orders = filled_entry_orders.copy()
    submitted_exit_orders['trip'] = 'EXIT'
    submitted_exit_orders['action'] = 'SELL'
    submitted_exit_orders['price'] = submitted_exit_orders['price'] * (1 + alpha2)
    submitted_exit_orders['status'] = 'SUBMITTED'

    # Figure out what happened to each exit order we submitted
    exit_order_fates = submitted_exit_orders.copy()
    exit_mkt_orders = pd.DataFrame(columns=exit_order_fates.columns)

    for index, exit_order in submitted_exit_orders.iterrows():

        # was it filled the day it was submitted?
        if float(
                ivv_prc.loc[ivv_prc['Date'] == exit_order['date'], 'Close Price']
        ) >= exit_order['price']:
            exit_order_fates.at[index, 'status'] = 'FILLED'
            continue

        window_prices = ivv_prc[ivv_prc['Date'] > exit_order['date']].head(n2)

        # was it submitted on the very last day for which we have data?
        if window_prices.size == 0:
            exit_order_fates.at[index, 'date'] = pd.to_datetime(
                next_business_day).date()

            exit_order_fates.at[index, 'status'] = 'LIVE'
            continue

        filled_ind, *asdf = np.where(
            window_prices['High Price'] >= exit_order['price']
        )

        if filled_ind.size == 0:

            if window_prices.shape[0] < n2:
                exit_order_fates.at[index, 'date'] = pd.to_datetime(
                    next_business_day).date()

                exit_order_fates.at[index, 'status'] = 'LIVE'
                continue

            exit_order_fates.at[index, 'date'] = window_prices['Date'].iloc[
                window_prices.shape[0] - 1
                ]
            exit_order_fates.at[index, 'status'] = 'CANCELLED'
            exit_mkt_orders = pd.concat([
                exit_mkt_orders,
                pd.DataFrame({
                    'trade_id': exit_order['trade_id'],
                    'date': window_prices['Date'].tail(1),
                    'asset': exit_order['asset'],
                    'trip': exit_order['trip'],
                    'action': exit_order['action'],
                    'type': "MKT",
                    'price': window_prices['Close Price'].tail(1),
                    'status': 'FILLED'
                })
            ])
            continue

        exit_order_fates.at[index, 'date'] = window_prices['Date'].iloc[
                min(filled_ind)
        ]
        exit_order_fates.at[index, 'status'] = 'FILLED'


    blotter = pd.concat(
        [entry_orders, submitted_exit_orders, exit_order_fates, exit_mkt_orders]
    ).sort_values(['trade_id', "date", 'trip']).reset_index(drop=True)

    return blotter

In [8]:
def blotter_to_ledger(blotter):
    ledger = pd.DataFrame(columns=['trade_id', 'asset', 'dt_enter', 'dt_exit', 'success', 'n', 'rtn'])

    # Calculate the length of the trade, use numpy built-in function count business day, but need to convert the date to <M8[D] type first.
    def calculate_n(start_date, end_date):
        start_day = pd.to_datetime(orders[start_date]['date'].values[0]).to_numpy().astype('<M8[D]')
        end_day = pd.to_datetime(orders[end_date]['date'].values[0]).to_numpy().astype('<M8[D]')
        return np.busday_count(start_day, end_day) + 1

    def calculate_rtn(start_price, end_price, n):
        return math.log(end_price / start_price) / n

    for trade_id in blotter['trade_id'].unique():
        orders = blotter[blotter['trade_id'] == trade_id]
        is_submitted_entry_order = ((orders['trip'] == 'ENTER') & (orders['status'] == 'SUBMITTED'))
        is_filled_entry_order = ((orders['trip'] == 'ENTER') & (orders['status'] == 'FILLED'))
        is_cancelled_entry_order = ((orders['trip'] == 'ENTER') & (orders['status'] == 'CANCELLED'))
        is_filled_exit_lmt_order = (
                    (orders['type'] == 'LMT') & (orders['trip'] == 'EXIT') & (orders['status'] == 'FILLED'))
        is_cancelled_exit_lmt_order = (
                    (orders['trip'] == 'EXIT') & (orders['type'] == 'LMT') & (orders['status'] == 'CANCELLED'))
        is_filled_exit_order = ((orders['trip'] == 'EXIT') & (orders['status'] == 'FILLED'))

        # Check if both of the entry order and the exit limit order are filled. If so, set success to 1, calculate the length of the trade and the rtn
        if any(is_filled_entry_order) and any(is_filled_exit_lmt_order):
            n = calculate_n(is_filled_entry_order, is_filled_exit_lmt_order)
            rtn = calculate_rtn(orders[is_filled_entry_order]['price'].values[0],
                                orders[is_filled_exit_lmt_order]['price'].values[0], n)
            success = 1

        # Check if the entry order is filled and the exit limit order is cancelled. If so, set success to -1, calculate the length of the trade and the rtn. The rtn should use the price of market order.
        elif any(is_filled_entry_order) and any(is_cancelled_exit_lmt_order):
            n = calculate_n(is_filled_entry_order, is_cancelled_exit_lmt_order)
            rtn = calculate_rtn(orders[is_filled_entry_order]['price'].values[0],
                                orders[is_filled_exit_order]['price'].values[0], n)
            success = -1

        # Check if the entry order is cancelled. If so, set success to 0
        elif any(is_cancelled_entry_order):
            n = calculate_n(is_submitted_entry_order, is_cancelled_entry_order)
            rtn = None
            success = 0

        # Set success to null in any other cases
        else:
            n = None
            rtn = None
            success = None

        # Get the asset dt_enter, dt_exit, and the length of the trade.
        asset = orders[orders['trip'] == 'ENTER']['asset'].values[0]
        date_enter = orders[orders['trip'] == 'ENTER']['date'].values[0]
        if any(is_filled_exit_order):
            date_exit = orders[is_filled_exit_order]['date'].values[0]
        else:
            date_exit = None

        # Concat the new record to the ledger
        ledger = pd.concat([ledger, pd.DataFrame({'trade_id': [trade_id],
                                                  'asset': [asset],
                                                  'dt_enter': [date_enter],
                                                  'dt_exit': [date_exit],
                                                  'success': [success],
                                                  'n': [n],
                                                  'rtn': [rtn]})],
                           ignore_index=True)

    return ledger

In [13]:
query_refinitiv()

    Instrument  Open Price  High Price  Low Price  Close Price        Date
0          IVV      324.98      326.32     323.95       326.32  2020-01-02
1          IVV      322.53      325.06    322.511       323.81  2020-01-03
2          IVV      321.89      325.14     321.78       325.09  2020-01-06
3          IVV      324.46      324.97     323.65        324.2  2020-01-07
4          IVV      324.38      327.21     324.11       325.85  2020-01-08
..         ...         ...         ...        ...          ...         ...
821        IVV      408.57     411.265     407.44       411.01  2023-04-06
822        IVV      408.37       411.5     407.76       411.48  2023-04-10
823        IVV      412.02      412.97     410.69       411.47  2023-04-11
824        IVV      413.68      413.94    409.225       409.93  2023-04-12
825        IVV      411.04      415.62      410.5       415.27  2023-04-13

[826 rows x 6 columns]


In [22]:
prices = query_refinitiv()
blotter = render_blotter(prices)
ledger = blotter_to_ledger(blotter)

    trade_id asset    dt_enter     dt_exit success     n       rtn
0          1   IVV  2020-01-03  2020-01-08       1     4  0.002488
1          2   IVV  2020-01-06        None       0     3      None
2          3   IVV  2020-01-07        None       0     3      None
3          4   IVV  2020-01-08        None       0     3      None
4          5   IVV  2020-01-09        None       0     3      None
..       ...   ...         ...         ...     ...   ...       ...
821      822   IVV  2023-04-10        None       0     3      None
822      823   IVV  2023-04-11        None       0     3      None
823      824   IVV  2023-04-12        None    None  None      None
824      825   IVV  2023-04-13        None    None  None      None
825      826   IVV  2023-04-14        None    None  None      None

[826 rows x 7 columns]


In [157]:
def process_feature(ledger):
    df1 = pd.read_excel('hw4_data.xlsx',sheet_name = 'PRICES_DATA')
    df2 = pd.read_excel('hw4_data.xlsx', sheet_name='IVV_DATA')
    prc = ledger
    # Rename the columns
    df1 = df1.rename(columns={
    'Dates': 'Date',
    'IVV US Equity': 'IVV_US_Equity',
    'IVV AU Equity': 'IVV_AU_Equity',
    'ECRPUS 1Y Index': 'ECRPUS_1Y_Index',
    'SPXSFRCS Index': 'SPXSFRCS_Index',
    'FDTRFTRL Index': 'FDTRFTRL_Index',
    'USCRWTIC Index': 'USCRWTIC_Index',
    'XAU Curncy': 'XAU_Curncy',
    'JPYUSD Curncy': 'JPYUSD_Curncy',
    'DXY Curncy': 'DXY_Curncy',
    'VIX Index': 'VIX_Index'
    })
    df2 = df2.rename(columns={
    'Dates': 'Date'
    })
    prc = prc.rename(columns={
    'dt_enter': 'Date',
    })
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1.loc[78, 'USCRWTIC_Index'] = 1
    prc['Date'] = pd.to_datetime(prc['Date'])

    merged_df = pd.merge(df1, df2, on='Date', how='outer')
    merged_df.drop('FDTRFTRL_Index', axis=1, inplace=True)
    #merged_df.dropna(subset=['success'], inplace=True)
    #merged_df = pd.merge(merged_df, prc, on='Date', how='outer')
    #merged_df['Date'] = merged_df['Date'].dt.strftime('%Y-%m-%d')
    exc_list = ['IVV_AU_Equity','IVV_AU_split','Date','IVOL_IMPLIED_FORWARD',
       'IVOL_DELTA']
    for col in merged_df.columns:
        if col not in exc_list:
            merged_df[f'{col}_log_return'] = np.log(merged_df[col]/merged_df[col].shift(1))

    merged_df['AU_log_return'] = np.log(merged_df['IVV_AU_Equity'] *(1/merged_df['IVV_AU_split'])/ merged_df['IVV_AU_Equity'].shift(1))
    merged_df = pd.merge(merged_df, prc, on='Date', how='outer')
    merged_df.dropna(subset=['success'], inplace=True)
    #merged_df['Date'] = merged_df['Date'].dt.strftime('%Y-%m-%d')
    
    merged_df = merged_df[['Date','success','IVV_US_Equity_log_return', 'ECRPUS_1Y_Index_log_return',
       'SPXSFRCS_Index_log_return', 'USCRWTIC_Index_log_return',
       'XAU_Curncy_log_return', 'JPYUSD_Curncy_log_return',
       'DXY_Curncy_log_return', 'VIX_Index_log_return',
       'AU_log_return','IVOL_IMPLIED_FORWARD',
       'IVOL_DELTA']]
    return merged_df


In [206]:
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler
def predict(model,X,y,X_test):
    sc = StandardScaler()
    sc.fit(X.values)
    X_std = sc.transform(X.values)
    X_test_std = sc.transform(np.array(X_test.values).reshape(1, -1))
    model.fit(X_std,y)
    y_pred = model.predict(X_test_std)
    return int(y_pred[0])

In [209]:
def new_ledger(features,n3):
    X = features.drop(['Date', 'success'], axis=1)
    y = features['success']
    prediction = []
    for i in range(len(X)-n3):
        X_piece = X.iloc[i:n3+i]
        X_test = X.iloc[n3+i]
        y_piece = y.iloc[i:n3+i]
        model = Perceptron(eta0=0.1)
        y_pred = predict(model, X_piece, np.asarray(y_piece, dtype="|S6"), X_test)
        prediction.append(y_pred)
    return prediction
features = process_feature(ledger)
ledger.dropna(subset=['success'], inplace=True)
ledger['new_success']=0
ledger['new_success'].iloc[50:] = new_ledger(features,50)
ledger[ledger['new_success']==1].head(50)

/var/folders/1n/6zcygvs12455myprvytb14680000gn/T/ipykernel_25054/3345915114.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ledger['new_success'].iloc[50:] = new_ledger(features,50)


,trade_id,asset,dt_enter,dt_exit,success,n,rtn,new_success
52,53,IVV,2020-03-19,2020-03-19,1,1,0.00995,1
54,55,IVV,2020-03-23,2020-03-24,1,2,0.004975,1
55,56,IVV,2020-03-24,None,0,3,None,1
57,58,IVV,2020-03-26,None,0,3,None,1
58,59,IVV,2020-03-27,2020-03-30,1,2,0.004975,1
59,60,IVV,2020-03-30,2020-04-06,1,4,0.002488,1
60,61,IVV,2020-03-31,2020-04-06,1,5,0.00199,1
61,62,IVV,2020-04-01,2020-04-06,1,4,0.002488,1
62,63,IVV,2020-04-02,None,0,3,None,1
64,65,IVV,2020-04-06,None,0,3,None,1


In [147]:
features = process_feature()
features

,Date,IVV_US_Equity_log_return,ECRPUS_1Y_Index_log_return,SPXSFRCS_Index_log_return,USCRWTIC_Index_log_return,XAU_Curncy_log_return,JPYUSD_Curncy_log_return,DXY_Curncy_log_return,VIX_Index_log_return,AU_log_return,IVOL_IMPLIED_FORWARD,IVOL_DELTA
2,2020-01-03,-0.007722,0.0,0.0,0.030108,0.014974,0.004225,-0.000083,0.117159,0.004289,323.2867,14.6881
3,2020-01-06,0.003945,0.0,0.0,0.003483,0.008685,-0.002381,-0.001726,-0.012200,0.001247,323.8963,14.6475
4,2020-01-07,-0.002741,0.0,0.0,-0.009050,0.005497,-0.000650,0.003449,-0.004342,0.009238,323.4212,14.6017
5,2020-01-08,0.005077,0.0,0.0,-0.050538,-0.011467,-0.006309,0.003026,-0.024965,0.000915,325.2006,13.9187
6,2020-01-09,0.006729,0.0,0.0,-0.000839,-0.002638,-0.003717,0.001551,-0.070056,0.011565,327.4236,13.6171
...,...,...,...,...,...,...,...,...,...,...,...,...
851,2023-04-06,0.003778,0.0,0.0,0.001116,-0.006364,-0.003552,-0.000295,-0.036290,0.001476,428.8891,17.8277
853,2023-04-10,0.001143,0.0,0.0,-0.011967,-0.008216,-0.013667,0.007397,0.030508,0.000000,430.2437,17.6069
854,2023-04-11,-0.000024,0.0,0.0,0.022200,0.006077,-0.000668,-0.003653,0.006830,0.009787,430.8804,17.6191
855,2023-04-12,-0.003750,0.0,0.0,0.020997,0.005629,0.004136,-0.006912,-0.000524,0.001460,428.7954,17.7061


In [148]:
ledger

,trade_id,asset,dt_enter,dt_exit,success,n,rtn
0,1,IVV,2020-01-03,2020-01-08,1,4,0.002488
1,2,IVV,2020-01-06,None,0,3,None
2,3,IVV,2020-01-07,None,0,3,None
3,4,IVV,2020-01-08,None,0,3,None
4,5,IVV,2020-01-09,None,0,3,None
...,...,...,...,...,...,...,...
821,822,IVV,2023-04-10,None,0,3,None
822,823,IVV,2023-04-11,None,0,3,None
823,824,IVV,2023-04-12,None,None,None,None
824,825,IVV,2023-04-13,None,None,None,None


In [162]:
a = [1,2,3]
a[1:]

[2, 3]